# California Housing Price Prediction

**Author:** Anik Tahabilder  
**Project:** 5 of 22 - Kaggle ML Portfolio  
**Dataset:** California Housing  
**Difficulty:** 4/10 | **Learning Value:** 8/10

---

## Project Overview

Predict **median house values** in California districts using census data. This is a classic **regression problem** - predicting a continuous value.

### What You'll Learn:

| Topic | Description |
|-------|-------------|
| **Data Exploration** | Understand geographic and demographic features |
| **Handling Missing Values** | Imputation strategies |
| **Feature Engineering** | Create meaningful derived features |
| **Multiple Regression Models** | Compare Linear, Ridge, Lasso, Random Forest, etc. |
| **Evaluation Metrics** | MSE, RMSE, MAE, R² for regression |

### Dataset Features:

| Feature | Description |
|---------|-------------|
| longitude/latitude | Geographic coordinates |
| housing_median_age | Median age of houses in block |
| total_rooms | Total rooms in block |
| total_bedrooms | Total bedrooms in block |
| population | Population in block |
| households | Number of households |
| median_income | Median income (scaled) |
| ocean_proximity | Location relative to ocean |
| **median_house_value** | **TARGET** - Median house value |

---

## Table of Contents

1. [Part 1: Setup and Data Loading](#part1)
2. [Part 2: Exploratory Data Analysis](#part2)
3. [Part 3: Data Preprocessing](#part3)
4. [Part 4: Feature Engineering](#part4)
5. [Part 5: Model Training](#part5)
6. [Part 6: Model Evaluation](#part6)
7. [Part 7: Feature Importance](#part7)
8. [Part 8: Summary and Conclusions](#part8)

---

<a id='part1'></a>
# Part 1: Setup and Data Loading

---

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

# Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:,.2f}'.format)

print("Libraries loaded successfully!")
print(f"scikit-learn version: {__import__('sklearn').__version__}")

In [ ]:
# Load the dataset
df = pd.read_csv('housing.csv')

print("=" * 60)
print("CALIFORNIA HOUSING DATASET")
print("=" * 60)
print(f"\nShape: {df.shape[0]:,} samples, {df.shape[1]} features")
print(f"\nColumns: {list(df.columns)}")
df.head(10)

In [ ]:
# Data info
print("=" * 60)
print("DATA INFO")
print("=" * 60)
df.info()

In [ ]:
# Statistical summary
print("=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
df.describe()

---

<a id='part2'></a>
# Part 2: Exploratory Data Analysis

---

## 2.1 Missing Values

In [ ]:
# Check missing values
print("=" * 60)
print("MISSING VALUES")
print("=" * 60)

missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Percent': missing_pct})
print(missing_df[missing_df['Missing'] > 0])

print(f"\nTotal missing: {missing.sum()} ({missing.sum()/len(df)*100:.2f}%)")
print("\nOnly total_bedrooms has missing values - will impute with median.")

## 2.2 Target Variable Distribution

In [ ]:
# Target variable: median_house_value
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Histogram
axes[0].hist(df['median_house_value'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(df['median_house_value'].mean(), color='red', linestyle='--', label=f"Mean: ${df['median_house_value'].mean():,.0f}")
axes[0].axvline(df['median_house_value'].median(), color='green', linestyle='--', label=f"Median: ${df['median_house_value'].median():,.0f}")
axes[0].set_xlabel('Median House Value ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of House Values', fontweight='bold')
axes[0].legend()

# Box plot
axes[1].boxplot(df['median_house_value'], vert=True)
axes[1].set_ylabel('Median House Value ($)')
axes[1].set_title('Box Plot of House Values', fontweight='bold')

# Log-transformed
axes[2].hist(np.log1p(df['median_house_value']), bins=50, edgecolor='black', alpha=0.7, color='darkorange')
axes[2].set_xlabel('Log(Median House Value)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Log-Transformed Distribution', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Right-skewed distribution")
print("- Cap at $500,001 (data censored)")
print(f"- Range: ${df['median_house_value'].min():,.0f} - ${df['median_house_value'].max():,.0f}")

## 2.3 Geographic Distribution

In [ ]:
# Geographic visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Population density
scatter1 = axes[0].scatter(df['longitude'], df['latitude'], 
                           c=df['population'], cmap='viridis',
                           alpha=0.4, s=df['population']/100)
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_title('California: Population Density', fontweight='bold')
plt.colorbar(scatter1, ax=axes[0], label='Population')

# House values
scatter2 = axes[1].scatter(df['longitude'], df['latitude'], 
                           c=df['median_house_value'], cmap='jet',
                           alpha=0.4, s=df['population']/100)
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].set_title('California: House Values', fontweight='bold')
plt.colorbar(scatter2, ax=axes[1], label='Median House Value ($)')

plt.tight_layout()
plt.show()

print("\nGeographic Insights:")
print("- Coastal areas (especially Bay Area, LA) have highest values")
print("- Population concentrated along coast")
print("- Interior areas generally cheaper")

## 2.4 Ocean Proximity Analysis

In [ ]:
# Ocean proximity analysis
print("=" * 60)
print("OCEAN PROXIMITY CATEGORIES")
print("=" * 60)
print(df['ocean_proximity'].value_counts())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count
ocean_counts = df['ocean_proximity'].value_counts()
axes[0].bar(ocean_counts.index, ocean_counts.values, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Ocean Proximity')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Ocean Proximity', fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

# House values by ocean proximity
df.boxplot(column='median_house_value', by='ocean_proximity', ax=axes[1])
axes[1].set_xlabel('Ocean Proximity')
axes[1].set_ylabel('Median House Value ($)')
axes[1].set_title('House Values by Ocean Proximity', fontweight='bold')
plt.suptitle('')  # Remove automatic title
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Mean values by ocean proximity
print("\nMean House Value by Ocean Proximity:")
print(df.groupby('ocean_proximity')['median_house_value'].mean().sort_values(ascending=False))

## 2.5 Feature Distributions

In [ ]:
# Distribution of numerical features
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('median_house_value')  # Remove target

fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(numerical_cols):
    axes[i].hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[i].set_xlabel(col)
    axes[i].set_title(f'Distribution of {col}', fontweight='bold')

# Hide empty subplot
if len(numerical_cols) < 9:
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()

## 2.6 Correlation Analysis

In [ ]:
# Correlation matrix
fig, ax = plt.subplots(figsize=(12, 10))

corr_matrix = df.select_dtypes(include=[np.number]).corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0,
            fmt='.2f', linewidths=0.5, mask=mask,
            annot_kws={'size': 10})
plt.title('Feature Correlation Matrix', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

# Correlations with target
print("\nCorrelations with median_house_value:")
target_corr = corr_matrix['median_house_value'].sort_values(ascending=False)
print(target_corr)

In [ ]:
# Scatter plots of top correlated features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

top_features = ['median_income', 'total_rooms', 'housing_median_age', 'latitude']

for i, feat in enumerate(top_features):
    ax = axes[i // 2, i % 2]
    ax.scatter(df[feat], df['median_house_value'], alpha=0.3, s=5)
    ax.set_xlabel(feat)
    ax.set_ylabel('Median House Value ($)')
    ax.set_title(f'{feat} vs House Value (r={corr_matrix.loc[feat, "median_house_value"]:.2f})', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nKey Insight: Median income has the STRONGEST correlation with house value!")

---

<a id='part3'></a>
# Part 3: Data Preprocessing

---

## 3.1 Handle Missing Values

In [ ]:
# Create a clean copy
data = df.copy()

# Impute total_bedrooms with median
print("Handling missing values...")
print(f"  total_bedrooms missing before: {data['total_bedrooms'].isnull().sum()}")

data['total_bedrooms'] = data['total_bedrooms'].fillna(data['total_bedrooms'].median())

print(f"  total_bedrooms missing after: {data['total_bedrooms'].isnull().sum()}")
print("\nMissing values handled!")

## 3.2 Handle Outliers

In [ ]:
# Check for capped values in target
print("=" * 60)
print("CAPPED VALUES CHECK")
print("=" * 60)

capped = (data['median_house_value'] >= 500000).sum()
print(f"\nHouses at $500,001 cap: {capped} ({capped/len(data)*100:.1f}%)")
print("\nNote: These are censored values - actual prices could be higher.")
print("For this tutorial, we'll keep them as-is.")

---

<a id='part4'></a>
# Part 4: Feature Engineering

---

Create new features that might be more predictive than raw values.

In [ ]:
# Feature Engineering
print("=" * 60)
print("FEATURE ENGINEERING")
print("=" * 60)

# 1. Rooms per household
data['rooms_per_household'] = data['total_rooms'] / data['households']
print("\n1. Created 'rooms_per_household' = total_rooms / households")

# 2. Bedrooms per room
data['bedrooms_per_room'] = data['total_bedrooms'] / data['total_rooms']
print("2. Created 'bedrooms_per_room' = total_bedrooms / total_rooms")

# 3. Population per household
data['population_per_household'] = data['population'] / data['households']
print("3. Created 'population_per_household' = population / households")

# 4. Bedrooms per household
data['bedrooms_per_household'] = data['total_bedrooms'] / data['households']
print("4. Created 'bedrooms_per_household' = total_bedrooms / households")

# 5. Income category
data['income_category'] = pd.cut(data['median_income'],
                                  bins=[0, 1.5, 3, 4.5, 6, np.inf],
                                  labels=[1, 2, 3, 4, 5])
data['income_category'] = data['income_category'].astype(int)
print("5. Created 'income_category' (1-5 buckets)")

print(f"\nTotal features now: {data.shape[1]}")

In [ ]:
# Visualize engineered features
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

engineered = ['rooms_per_household', 'bedrooms_per_room', 'population_per_household', 'income_category']

for i, feat in enumerate(engineered):
    ax = axes[i // 2, i % 2]
    ax.scatter(data[feat], data['median_house_value'], alpha=0.3, s=5)
    ax.set_xlabel(feat)
    ax.set_ylabel('Median House Value ($)')
    corr = data[feat].corr(data['median_house_value'])
    ax.set_title(f'{feat} vs House Value (r={corr:.2f})', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nEngineered Feature Correlations with Target:")
for feat in engineered:
    corr = data[feat].corr(data['median_house_value'])
    print(f"  {feat}: {corr:.3f}")

## 4.2 Prepare Features and Target

In [ ]:
# Define features and target
print("=" * 60)
print("FEATURE SELECTION")
print("=" * 60)

# Select features (drop target and original columns that we engineered from)
feature_cols = [
    'longitude', 'latitude', 'housing_median_age',
    'total_rooms', 'total_bedrooms', 'population', 'households',
    'median_income', 'ocean_proximity',
    'rooms_per_household', 'bedrooms_per_room', 
    'population_per_household', 'bedrooms_per_household', 'income_category'
]

X = data[feature_cols].copy()
y = data['median_house_value'].copy()

print(f"\nFeatures: {len(feature_cols)}")
print(f"  Numerical: {len(X.select_dtypes(include=[np.number]).columns)}")
print(f"  Categorical: {len(X.select_dtypes(include=['object']).columns)}")
print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")

## 4.3 Train-Test Split

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("=" * 60)
print("TRAIN-TEST SPLIT")
print("=" * 60)
print(f"\nTraining set: {len(X_train):,} samples ({len(X_train)/len(X)*100:.0f}%)")
print(f"Testing set: {len(X_test):,} samples ({len(X_test)/len(X)*100:.0f}%)")

## 4.4 Preprocessing Pipeline

In [ ]:
# Identify column types
numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"Numerical features ({len(numerical_features)}): {numerical_features}")
print(f"\nCategorical features ({len(categorical_features)}): {categorical_features}")

In [ ]:
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ]
)

# Fit and transform
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("=" * 60)
print("PREPROCESSING COMPLETE")
print("=" * 60)
print(f"\nX_train shape after preprocessing: {X_train_processed.shape}")
print(f"X_test shape after preprocessing: {X_test_processed.shape}")

# Get feature names after one-hot encoding
cat_encoder = preprocessor.named_transformers_['cat']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_features).tolist()
all_feature_names = numerical_features + cat_feature_names
print(f"\nTotal features after encoding: {len(all_feature_names)}")

---

<a id='part5'></a>
# Part 5: Model Training

---

## Regression Algorithms

| Algorithm | Type | Characteristics |
|-----------|------|----------------|
| **Linear Regression** | Linear | Simple, interpretable |
| **Ridge** | Linear + L2 | Handles multicollinearity |
| **Lasso** | Linear + L1 | Feature selection |
| **Decision Tree** | Tree-based | Non-linear, interpretable |
| **Random Forest** | Ensemble | Robust, handles overfitting |
| **Gradient Boosting** | Ensemble | High accuracy |

In [ ]:
# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=1.0, random_state=42),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
}

print("Models to train:")
for i, name in enumerate(models.keys(), 1):
    print(f"  {i}. {name}")

In [ ]:
# Train all models
results = {}

print("=" * 70)
print("TRAINING MODELS")
print("=" * 70)

for name, model in models.items():
    # Train
    model.fit(X_train_processed, y_train)
    
    # Predict
    y_pred_train = model.predict(X_train_processed)
    y_pred_test = model.predict(X_test_processed)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    
    # Store results
    results[name] = {
        'model': model,
        'predictions': y_pred_test,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'test_r2': test_r2
    }
    
    print(f"\n{name}:")
    print(f"  Train RMSE: ${train_rmse:,.0f}")
    print(f"  Test RMSE:  ${test_rmse:,.0f}")
    print(f"  Test MAE:   ${test_mae:,.0f}")
    print(f"  Test R²:    {test_r2:.4f}")

print("\n" + "=" * 70)
print("All models trained successfully!")

---

<a id='part6'></a>
# Part 6: Model Evaluation

---

## 6.1 Model Comparison

In [ ]:
# Create comparison table
comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'Train RMSE': [r['train_rmse'] for r in results.values()],
    'Test RMSE': [r['test_rmse'] for r in results.values()],
    'Test MAE': [r['test_mae'] for r in results.values()],
    'R²': [r['test_r2'] for r in results.values()]
}).sort_values('Test RMSE').reset_index(drop=True)

comparison['Rank'] = range(1, len(comparison) + 1)
comparison = comparison[['Rank', 'Model', 'Train RMSE', 'Test RMSE', 'Test MAE', 'R²']]

print("=" * 80)
print("MODEL COMPARISON (sorted by Test RMSE)")
print("=" * 80)
print(comparison.to_string(index=False))

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sort by test RMSE
sorted_results = dict(sorted(results.items(), key=lambda x: x[1]['test_rmse']))
names = list(sorted_results.keys())
rmses = [sorted_results[n]['test_rmse'] for n in names]
r2s = [sorted_results[n]['test_r2'] for n in names]

colors = plt.cm.RdYlGn(np.linspace(0.9, 0.3, len(names)))  # Green = better (lower RMSE)

# RMSE
bars1 = axes[0].barh(names[::-1], rmses[::-1], color=colors[::-1], edgecolor='black')
axes[0].set_xlabel('Test RMSE ($)')
axes[0].set_title('Model Comparison: RMSE (Lower is Better)', fontweight='bold')
for bar, rmse in zip(bars1, rmses[::-1]):
    axes[0].text(bar.get_width() + 1000, bar.get_y() + bar.get_height()/2,
                 f'${rmse:,.0f}', va='center', fontweight='bold')

# R²
colors_r2 = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(names)))  # Green = better (higher R²)
bars2 = axes[1].barh(names[::-1], r2s[::-1], color=colors_r2[::-1], edgecolor='black')
axes[1].set_xlabel('R² Score')
axes[1].set_xlim(0, 1)
axes[1].set_title('Model Comparison: R² (Higher is Better)', fontweight='bold')
for bar, r2 in zip(bars2, r2s[::-1]):
    axes[1].text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                 f'{r2:.3f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

best_model = names[0]
print(f"\nBest Model: {best_model}")
print(f"  Test RMSE: ${sorted_results[best_model]['test_rmse']:,.0f}")
print(f"  Test R²: {sorted_results[best_model]['test_r2']:.4f}")

## 6.2 Cross-Validation

In [ ]:
# 5-Fold Cross-Validation
print("=" * 70)
print("5-FOLD CROSS-VALIDATION (using negative RMSE)")
print("=" * 70)

cv_results = {}

for name, model in models.items():
    # Cross-validation with negative MSE (sklearn convention)
    scores = cross_val_score(model, X_train_processed, y_train, 
                             cv=5, scoring='neg_root_mean_squared_error')
    rmse_scores = -scores  # Convert to positive RMSE
    
    cv_results[name] = {
        'mean': rmse_scores.mean(),
        'std': rmse_scores.std(),
        'scores': rmse_scores
    }
    
    print(f"\n{name}:")
    print(f"  RMSE scores: {rmse_scores.round(0)}")
    print(f"  Mean RMSE: ${rmse_scores.mean():,.0f} (+/- ${rmse_scores.std()*2:,.0f})")

In [ ]:
# Visualize CV results
cv_sorted = dict(sorted(cv_results.items(), key=lambda x: x[1]['mean']))

fig, ax = plt.subplots(figsize=(12, 6))

names_cv = list(cv_sorted.keys())
means = [cv_sorted[n]['mean'] for n in names_cv]
stds = [cv_sorted[n]['std'] for n in names_cv]

colors = plt.cm.RdYlGn(np.linspace(0.9, 0.3, len(names_cv)))
bars = ax.barh(names_cv[::-1], means[::-1], xerr=stds[::-1],
               color=colors[::-1], edgecolor='black', capsize=5)

ax.set_xlabel('Cross-Validation RMSE ($)')
ax.set_title('5-Fold Cross-Validation Results', fontweight='bold', fontsize=14)

for bar, mean, std in zip(bars, means[::-1], stds[::-1]):
    ax.text(bar.get_width() + std + 1000, bar.get_y() + bar.get_height()/2,
            f'${mean:,.0f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 6.3 Actual vs Predicted Plots

In [ ]:
# Actual vs Predicted for top models
top_models = list(sorted_results.keys())[:4]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for i, name in enumerate(top_models):
    y_pred = results[name]['predictions']
    
    axes[i].scatter(y_test, y_pred, alpha=0.3, s=10)
    axes[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                 'r--', linewidth=2, label='Perfect Prediction')
    axes[i].set_xlabel('Actual House Value ($)')
    axes[i].set_ylabel('Predicted House Value ($)')
    axes[i].set_title(f'{name}\nR² = {results[name]["test_r2"]:.4f}', fontweight='bold')
    axes[i].legend()

plt.tight_layout()
plt.show()

print("Points closer to the red diagonal line = better predictions")

## 6.4 Residual Analysis

In [ ]:
# Residual plots for best model
best_model_name = list(sorted_results.keys())[0]
best_predictions = results[best_model_name]['predictions']
residuals = y_test - best_predictions

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Residuals vs Predicted
axes[0].scatter(best_predictions, residuals, alpha=0.3, s=10)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted House Value ($)')
axes[0].set_ylabel('Residuals ($)')
axes[0].set_title('Residuals vs Predicted', fontweight='bold')

# Residual distribution
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuals ($)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution', fontweight='bold')

# Q-Q plot
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot (Normality Check)', fontweight='bold')

plt.suptitle(f'Residual Analysis: {best_model_name}', fontweight='bold', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(f"\nResidual Statistics for {best_model_name}:")
print(f"  Mean: ${residuals.mean():,.0f}")
print(f"  Std: ${residuals.std():,.0f}")
print(f"  Min: ${residuals.min():,.0f}")
print(f"  Max: ${residuals.max():,.0f}")

---

<a id='part7'></a>
# Part 7: Feature Importance

---

In [ ]:
# Feature importance from Random Forest
rf_model = results['Random Forest']['model']

feature_importance = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("=" * 60)
print("FEATURE IMPORTANCE (Random Forest)")
print("=" * 60)
print(feature_importance.head(15).to_string(index=False))

In [ ]:
# Visualize top 15 features
top_n = 15
top_features = feature_importance.head(top_n)

fig, ax = plt.subplots(figsize=(10, 8))

colors = plt.cm.viridis(np.linspace(0.3, 0.9, top_n))
bars = ax.barh(top_features['Feature'][::-1], 
               top_features['Importance'][::-1] * 100,
               color=colors[::-1], edgecolor='black')

ax.set_xlabel('Importance (%)', fontsize=12)
ax.set_title('Top 15 Features for House Price Prediction', fontweight='bold', fontsize=14)

for bar, imp in zip(bars, top_features['Importance'][::-1]):
    ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2,
            f'{imp*100:.1f}%', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nKey Insight: Median income is by far the most important predictor!")

In [ ]:
# Linear Regression coefficients
lr_model = results['Linear Regression']['model']

lr_importance = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

# Top 15
fig, ax = plt.subplots(figsize=(10, 8))

top_lr = lr_importance.head(15)
colors = ['green' if c > 0 else 'red' for c in top_lr['Coefficient']]

ax.barh(top_lr['Feature'][::-1], top_lr['Coefficient'][::-1], 
        color=colors[::-1], edgecolor='black', alpha=0.7)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Coefficient Value')
ax.set_title('Linear Regression Coefficients (Standardized)', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

print("Green = Positive effect on price")
print("Red = Negative effect on price")

---

<a id='part8'></a>
# Part 8: Summary and Conclusions

---

In [ ]:
# Summary dashboard
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# 1. Model Comparison
ax1 = fig.add_subplot(gs[0, :])
models_sorted = list(sorted_results.keys())
r2_sorted = [sorted_results[m]['test_r2'] for m in models_sorted]
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(models_sorted)))
bars = ax1.bar(models_sorted, r2_sorted, color=colors, edgecolor='black')
ax1.set_ylim(0, 1)
ax1.set_ylabel('R² Score')
ax1.set_title('Model R² Comparison', fontweight='bold', fontsize=14)
for bar, r2 in zip(bars, r2_sorted):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{r2:.3f}', ha='center', fontweight='bold')
ax1.tick_params(axis='x', rotation=15)

# 2. Feature Importance
ax2 = fig.add_subplot(gs[1, 0])
top_feat = feature_importance.head(8)
ax2.barh(top_feat['Feature'][::-1], 
         top_feat['Importance'][::-1] * 100,
         color=plt.cm.viridis(np.linspace(0.3, 0.9, 8))[::-1], edgecolor='black')
ax2.set_xlabel('Importance (%)')
ax2.set_title('Top Features', fontweight='bold')

# 3. Best Model Predictions
ax3 = fig.add_subplot(gs[1, 1])
ax3.scatter(y_test, best_predictions, alpha=0.3, s=10)
ax3.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
ax3.set_xlabel('Actual ($)')
ax3.set_ylabel('Predicted ($)')
ax3.set_title(f'Best Model: {best_model_name}', fontweight='bold')

# 4. Target Distribution
ax4 = fig.add_subplot(gs[1, 2])
ax4.hist(y, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax4.axvline(y.mean(), color='red', linestyle='--', label=f'Mean: ${y.mean():,.0f}')
ax4.set_xlabel('House Value ($)')
ax4.set_ylabel('Frequency')
ax4.set_title('Target Distribution', fontweight='bold')
ax4.legend()

plt.suptitle('CALIFORNIA HOUSING PREDICTION - SUMMARY DASHBOARD', 
             fontweight='bold', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

---

## Key Findings

### 1. Model Performance
- **Best model**: Random Forest / Gradient Boosting
- **R² scores**: 0.80-0.82 (explains ~80% of variance)
- **RMSE**: ~$50,000-60,000 prediction error

### 2. Most Important Features
| Feature | Importance |
|---------|------------|
| **median_income** | Highest - strong predictor |
| **Location (lat/long)** | Geographic factors matter |
| **ocean_proximity** | Coastal = more expensive |
| **housing_median_age** | Older areas have different values |

### 3. Key Insights
- **Income is king**: Median income is the strongest predictor
- **Location matters**: Coastal areas command premiums
- **Ensemble methods win**: Random Forest and Gradient Boosting outperform linear models
- **Feature engineering helps**: Derived features like rooms_per_household add value

---

## Regression Checklist

- [x] Load and explore data
- [x] Handle missing values
- [x] Visualize distributions and correlations
- [x] Engineer new features
- [x] Encode categorical variables
- [x] Scale features
- [x] Train multiple models
- [x] Evaluate with RMSE, MAE, R²
- [x] Cross-validate
- [x] Analyze residuals
- [x] Interpret feature importance

---

**End of California Housing Prediction Tutorial**

In [ ]:
# Final summary
print("="*70)
print("CALIFORNIA HOUSING PREDICTION - FINAL SUMMARY")
print("="*70)

print(f"\nDATASET")
print(f"   Samples: {len(data):,}")
print(f"   Features: {len(feature_cols)}")
print(f"   Target: median_house_value")
print(f"   Mean house value: ${y.mean():,.0f}")

print(f"\nBEST MODEL")
print(f"   Name: {best_model_name}")
print(f"   Test RMSE: ${results[best_model_name]['test_rmse']:,.0f}")
print(f"   Test MAE: ${results[best_model_name]['test_mae']:,.0f}")
print(f"   Test R²: {results[best_model_name]['test_r2']:.4f}")

print(f"\nALL MODEL R² SCORES")
for name in sorted_results:
    print(f"   {name}: {sorted_results[name]['test_r2']:.3f}")

print(f"\nTOP 5 FEATURES")
for _, row in feature_importance.head(5).iterrows():
    print(f"   {row['Feature']}: {row['Importance']*100:.1f}%")

print("\n" + "="*70)
print("PREDICTION COMPLETE!")
print("="*70)